# 4-2. LangChain Agent 만들기: RAG와 도구(Tools)의 결합
**실습 개요 및 시나리오**

이번 실습의 목표는 단순한 챗봇이 아닌, **'생각(Reason)'** 하고 **'행동(Act)'** 할 수 있는 AI 고객센터 직원을 만드는 것입니다.

**지난 시간 (`4-1_RAG.ipynb`) 복습:** 우리는 `shipping_policy.txt`와 같은 '비정형 데이터(PDF, TXT)'를 Vector Store에 저장하고, '검색(Retrieve)'을 통해 LLM이 문서에 근거한 답변을 하도록 **RAG 파이프라인**을 구축했습니다.

**이번 시간 (`4-2_Agent.ipynb`) 시나리오:** 고객센터 AI는 RAG(문서 검색)만으로는 모든 일을 처리할 수 없습니다. "내 주문 배송 상태 좀 알려줘"와 같은 요청은 '구조화된 데이터(DB)'를 조회해야 합니다.

1. **환경 설정:**
    - Agent가 사용할 실제 데이터베이스(`orders.db`)를 `sqlite3`로 만듭니다.
    - (참고: SQL 문법이 조금 나오지만, Agent가 실제 DB와 연동하는 것을 보여주기 위함입니다. 본과정 DB 파트에서 자세히 다룰 예정입니다.)

2. **도구(Tools) 준비:**
    - (Tool 1) 지난 시간에 만든 RAG 파이프라인을 Agent가 쓸 수 있는 '정책 검색 도구'로 포장합니다.
    - (Tool 2) `orders.db`를 조회하는 '주문 상태 확인 도구'를 만듭니다.
    - (Tool 3) 배송 지연 시 '쿠폰 발급 도구'를 만듭니다. (DB 조회 + 쿠폰 발급은 하드코딩)

3. **ReAct Agent 생성:**
    - LLM(두뇌)에 1, 2, 3번 도구(손/발)를 장착시키고, '시스템 프롬프트(규칙)'를 부여하여 Agent를 완성합니다.

4. **Agent 테스트 (T-A-O 로그):**
    - Agent가 사용자의 요청을 받고, 스스로 `'생각(Thought) -> 도구 선택(Action) -> 결과 관찰(Observation)'`을 반복하며 작업을 완료하는 **ReAct 사이클** 을 실시간 스트리밍으로 관찰합니다. (이 부분이 오늘 수업에서 가장 흥미로운 지점)

5. **(Extra) 대화 메모리 추가:**
    - 방금 나눈 대화를 기억하게 만들어, "방금 말한 그 주문"이 무엇인지 알아듣게 만듭니다. (`langgraph-checkpoint-sqlite` 활용)

## 0. 환경 설정

### 0-1. 라이브러리 설치

- RAG 파이프라인과 Agent 구현에 필요한 라이브러리를 설치합니다.
- LangChain은 1.0 버전 이후로 여러 패키지로 분리되었습니다. (core, community, upstage 등)

In [ ]:
# LangChain 1.0+ 설치 (핵심, 커뮤니티, 파트너 패키지 포함)
# langchain: Agent, LCEL(LangChain Expression Language) 등 핵심 기능
!pip install -U langchain
# langchain-core: LangChain의 기본 데이터 구조 (Schema, Runnable 등)
!pip install -U langchain-core
# langchain-community: ChromaDB, TextLoader 등 커뮤니티 기반 연동 모듈
!pip install -U langchain-community
# langchain-upstage: Upstage LLM, Embedding 모델 연동 패키지
# (구버전과 호환성 문제가 있을 수 있어 -U로 업그레이드 설치 권장)
!pip install -U langchain-upstage

# RAG 파이프라인 관련
# langchain-text-splitters: 문서를 청크로 분할하는 도구
!pip install langchain-text-splitters
# tiktoken: 텍스트를 토큰 단위로 계산하는 라이브러리
!pip install tiktoken

# Vector Store
# chromadb: 로컬(인메모리 또는 파일) 기반의 벡터 데이터베이스
!pip install chromadb

# Document Loaders (PDF 처리용)
!pip install pypdf pymupdf pypdfium2

  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.1
    Uninstalling langchain-core-1.0.1:
      Successfully uninstalled langchain-core-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-text-splitters 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.79 which is incompatible.
langchain 1.0.2 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
  Using cached langchai

### 중요. 설치 완료 후 버젼 확인 필수

- 1.0 버전이 올바르게 설치되었는지 확인합니다.
- 만약 구버전이 확인된다면, Colab 런타임 **[런타임] -> [세션 다시 시작]** 을 실행해 주세요.
- `restart` 후에도 캐시 데이터에 의해 구버전이 출력될 수 있습니다. 실습 중 버전 이슈가 발생하면 다시 `restart`를 시도합니다.

In [ ]:
# langchain과 langchain-core의 버전을 확인합니다.
# (1.0 이상이어야 함)
!pip show langchain
!pip show langchain-core

### 0-2. Upstage API Key 설정

- [Upstage API Docs](https://console.upstage.ai/docs/getting-started)
- dotenv 라이브러리를 활용하여 .env 파일에서 API 키를 안전하게 로드합니다.
    - **load_dotenv:** .env 파일에 저장된 KEY=VALUE 쌍을 환경 변수로 로드
    - **getenv:** 로드된 환경 변수 중 "UPSTAGE_API_KEY"에 해당하는 값을 읽음

In [ ]:
from google.colab import drive
from dotenv import load_dotenv
from pprint import pprint
from os import getenv

# 1. Google Drive 마운트 (드라이브에 저장된 .env 파일을 접근하기 위함)
drive.mount('/content/drive')

# 2. .env 파일이 저장된 기본 경로를 설정합니다.
# (본인의 구글 드라이브 환경에 맞게 이 경로를 수정해야 합니다.)
base_path = '/content/drive/MyDrive/Colab Notebooks/AI/11_Agent/'

Mounted at /content/drive


In [ ]:
# base_path에 지정된 디렉토리가 존재하지 않을 경우 생성 (-p 옵션).
# .env 파일을 저장할 폴더를 미리 준비함.
!mkdir -p '{base_path}'

In [1]:
# Colab 환경에서 .env 파일을 생성하고 API 키를 저장하는 명령어.
# {your_api_key} 부분에 본인의 실제 Upstage API 키를 입력해야 함.
# '!'는 Colab에서 셸 명령어를 실행함을 의미.
!echo 'UPSTAGE_API_KEY={your_api_key}' > '{base_path}.env'

# 기존에 만든.env 파일을 들고와서, 실행은 생략!

In [ ]:
# 3. .env 파일 로드
# base_path 경로에 있는 .env 파일을 찾아 환경 변수로 로드합니다.
# load_dotenv(".env") # 로컬 환경용
load_dotenv(base_path + ".env")

# 4. 환경 변수 읽기
# 런타임 환경에 로드된 'UPSTAGE_API_KEY' 값을 읽어옵니다.
UPSTAGE_API_KEY = getenv("UPSTAGE_API_KEY")

# API 키가 성공적으로 로드되었는지 확인하고 메시지를 출력.
if UPSTAGE_API_KEY:
    print('Success API Key Setting!')
else:
    print(f'ERROR: Failed to load UPSTAGE_API_KEY from {base_path}')

Success API Key Setting!


### 0-3. DB 생성 및 데이터 삽입

- Agent가 사용할 '도구(Tool)'는 하드코딩된 값보다 실제 데이터 소스에 연결될 때 훨씬 강력합니다.
- `sqlite3`를 사용해 "AI 온라인 서점"의 가상 주문 내역 데이터베이스(`orders.db`)를 생성합니다.
python으로 SQL 문법을 사용할 수 있도록 해주는 sqlite3 라이브러리 사용
- **(참고):** SQL 문법이 익숙하지 않아도 괜찮습니다. "Agent가 이런 DB를 조회할 수 있구나" 정도로 이해하고 넘어가셔도 됩니다.


In [ ]:
import sqlite3

# 1. DB 및 테이블 생성 함수 정의
def setup_refined_database():
    # base_path에 'orders.db'라는 파일 기반 DB를 생성(연결)
    conn = sqlite3.connect(base_path + 'orders.db')
    # 커서(Cursor)는 DB에 SQL 명령어를 실행하는 객체
    c = conn.cursor()
    try:
        # 'orders'라는 이름의 테이블(표) 생성
        # (order_id, customer_name, item_name, status 4개의 열을 가짐)
        c.execute('''
            CREATE TABLE orders (
                order_id TEXT PRIMARY KEY,
                customer_name TEXT,
                item_name TEXT,
                status TEXT
            )
        ''')
        # 샘플 데이터(가상 주문 내역) 삽입
        # Delivered         : 배송 완료
        # Shipping Delayed  : 배송 지연
        # Processing        : 처리 중
        c.executemany('INSERT INTO orders (order_id, customer_name, item_name, status) VALUES (?, ?, ?, ?)', [
            ('ORDER123', 'Alice Smith', 'Introduction to Python', 'Delivered'),
            ('ORDER456', 'Bob Johnson', 'The AI Revolution', 'Shipping Delayed'),
            ('ORDER789', 'Charlie Lee', 'Data Structures', 'Processing'),
            ('ORDER101', 'David Kim', 'Learning SQL', 'Processing'),
            ('ORDER102', 'Alice Smith', 'Advanced ML', 'Delivered'),
            ('ORDER505', 'Eva Moon', 'Cloud Computing', 'Shipping Delayed')
        ])
        # 변경사항을 DB 파일에 최종 저장(Commit)
        conn.commit()
        print("--- Mock 'orders.db' 생성 완료 ---")
    except sqlite3.OperationalError:
        # 테이블이 이미 존재하면(두 번째 실행 시) 이 메시지를 출력
        print("--- Mock 'orders.db'가 이미 존재합니다 ---")
    finally:
        # 작업이 끝나면 DB 연결 종료
        conn.close()

# DB 셋업 함수 실행
setup_refined_database()

--- Mock 'orders.db' 생성 완료 ---


## 1. RAG 및 Custom Tool: Agent의 '손발' 만들기

Agent는 '두뇌(LLM)'와 '손발(Tools)'로 구성됩니다. LLM이 아무리 똑똑해도, 외부 세계와 상호작용할 도구가 없으면 아무것도 할 수 없습니다.

AI 고객센터 직원에게 두 가지 종류의 '손발'을 만들어 줄 것입니다.

1. **RAG Tool (문서 검색):** "배송 정책 알려줘"와 같이 정해진 답이 없는 '비정형' 질문에 답하기 위한 도구. (지난 시간 복습)
2. **Custom Tool:** "ORDER123 배송 상태 알려줘"와 같이 명확한 '정형' 요청을 처리하기 위한 도구.

### 1-1. (복습) RAG (Retriever) 준비

- 지난 시간에 배운 RAG 파이프라인(Load -> Chunk -> Embed & Store -> Retrieve)을 구축하여 '정책 검색' 도구의 기반을 만듭니다.
- **collection:** 벡터 스토어 내에서 벡터들을 그룹화하는 단위입니다. (예: "배송 정책" 컬렉션, "반품 정책" 컬렉션 등)

In [ ]:
import glob
# langchain-community에서 로더(PDF, TXT) 임포트
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
# data chunking을 위한 텍스트 스플리터
from langchain_text_splitters import RecursiveCharacterTextSplitter
# langchain-community에서 Vector Store (Chroma) 임포트
from langchain_community.vectorstores import Chroma
# Upstage 임베딩 모델 임포트
from langchain_upstage import UpstageEmbeddings

# 1. Load: data 폴더의 모든 PDF와 shipping_policy.txt 파일을 로드
!unzip -o "{base_path}/data.zip" -d "{base_path}/data"
pdf_files = glob.glob(base_path + '/data/*.pdf') # PDF 파일 경로
documents = []
for pdf_filepath in pdf_files:
    loader = PyMuPDFLoader(pdf_filepath) # PyMuPDF로 PDF 로드
    documents.extend(loader.load())

# 배송 정책 TXT 파일 로드
shipping_loader = TextLoader(base_path + "shipping_policy.txt")
documents.extend(shipping_loader.load())
print(f"총 {len(documents)}개의 문서를 로드했습니다.")

# 2. Chunk: 로드된 문서를 200자 단위(overlap 20자)로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # 청크 크기
    chunk_overlap=20, # 중복 크기
)
chunks = text_splitter.split_documents(documents)
print(f"문서를 총 {len(chunks)}개의 청크로 분할했습니다.")

# 3. Embed & Store: Upstage 임베딩 모델로 청크를 벡터화하여 ChromaDB에 저장
embeddings = UpstageEmbeddings(model="embedding-query") # Upstage 임베딩 모델

# ChromaDB에 청크들을 임베딩하여 저장
# persist_directory: 벡터 데이터를 디스크에 저장할 경로
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=base_path + "chroma_db_v1"
)
print(f"Vector Store 생성 완료. 컬렉션 개수: {vectorstore._collection.count()}")

# 4. Retriever: 저장된 Vector Store를 '검색기(Retriever)'로 변환
retriever = vectorstore.as_retriever()
print("RAG Retriever가 준비되었습니다.")

Archive:  /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data.zip
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data/도서 품절 보상제도 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data/매장 픽업 서비스_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data/무료배송 추가적립_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data/영원한 YES포인트_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data/배송지연 보상제도_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data/신규 회원 혜택_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data/총알배송_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/11_Agent//data/제휴 할인카드_서비스 혜택 - 예스24.pdf  
총 9개의 문서를 로드했습니다.
문서를 총 81개의 청크로 분할했습니다.
Vector Store 생성 완료. 컬렉션 개수: 81
RAG Retriever가 준비되었습니다.


### 1-2. 기능 도구(Tool) 정의

이제 Agent가 사용할 3개의 도구를 만듭니다.

**도구 정의 시 핵심: '설명(Description)'**

Agent(LLM)는 함수 코드를 직접 읽지 못합니다. 대신 함수의 **이름(name)** 과 **설명(docstring, `"""..."""`)** 을 읽고 언제 이 도구를 써야 할지 판단합니다.

- **(중요)** 도구의 설명을 **"AI가 이해하기 쉽게, 명확하고 구체적으로"** 작성하는 것이 Agent 성능에 매우 중요

**도구 1: 정책 검색 (RAG) 도구**
- 위에서 만든 retriever를 Agent가 사용할 수 있는 Tool 객체로 포장합니다.

In [ ]:
from langchain_classic.tools.retriever import create_retriever_tool

# create_retriever_tool: Retriever 객체를 Agent용 Tool로 변환하는 헬퍼 함수
retriever_tool = create_retriever_tool(
    retriever, # 1. 사용할 검색기(Retriever)
    "Policy_Search", # 2. Agent가 부를 도구 이름 (공백X, 영어 권장)
    # 3. Agent가 이 도구의 용도를 파악할 수 있는 상세한 설명 (가장 중요!)
    """배송 정책, 보상 정책, 환불 정책 등 AI 온라인 서점의
    다양한 정책 문서에서 고객 질문과 관련된 정보를 검색합니다.
    (예: "배송 지연 시 보상 정책이 어떻게 되나요?")
    """,
)

In [ ]:
# (테스트) Agent에게 주기 전에 도구가 잘 작동하는지 수동으로 테스트
retriever_tool.invoke({"query": "주말에 배송 되나요?"})

'주문시간\n서울, 수도권, 충청권\n평일(월~금) 당일배송 종료 직후 ~ 22시까지 주문 시\n토요일 21시 ~ 일요일 15시까지 주문 시 (월요일 아침 7시 전 배송)\n그 외 지역\n평일(월~금) 당일배송 종료 직후 ~ 18시까지 주문 시\n일요일 0시 ~ 15시까지 주문 시 (월요일 아침 7시 전 배송)\n\n0~13시\n0~12시\n0~12시\n0~12시\n0~12시\n0~13시\n0~13시\n오늘\n토요일\n0~11시\n0~12시\n0~12시\n0~11시\n0~12시\n0~11시\n0~12시\n0~12시\n0~12시\n오늘\n사무실로 주문하시는 경우 퇴근 등으로 당일 배송이 안될 수 있습니다.\n오후 늦게라도 상품 수령이 가능한 주소를 입력해 주시면 더 원활한 당일배송이 가능합니다.\n\n당일배송은 공휴일(일요일 제외)에도 일부 지역에 배송이 가능합니다.\n하루배송 표시상품을 주문하시면 다음날에 받으실 수 있습니다.\n지역\n시간\n서울\n수도권(평택제외), 인천\n대전, 세종, 청주, 천안,\n아산, 평택\n광주\n대구\n경산, 구미\n울산, 창원, 포항\n부산, 김해\n원주, 춘천\n그 외 지역\n물류센터 사정에 따라 일부 조정될 수 있습니다.\n\n서울, 수도권(평택 제외), 인천 주문에 한하여 토요일 21시, 그 외 지역은 18시까지 주문하면 월요일 또는 화요일에 배송이 됩니다.\n예스24\n서비스/혜택\n총알배송\n아침배송 표시 상품을 오늘 밤 10시 전 주문하시면 내일 아침 7시 전에 받으실 수 있습니다.\n배송지역\n서울, 수도권, 충청권\n호남권 (광주, 군산, 익산, 전주)'

**도구 2: 주문 상태 조회 (DB 검색) 도구**
- 파이썬 함수를 `@tool` 데코레이터로 감싸면 간단하게 Agent용 Tool로 만들 수 있습니다.
- 이 도구는 `orders.db`에서 특정 주문 ID의 상태를 조회합니다.

In [ ]:
from langchain.tools import tool

# @tool: 이 데코레이터가 붙은 함수를 LangChain Tool로 자동 변환
@tool
def get_order_status_from_db(order_id: str) -> str:
    """(DB 조회)
        주문 ID를 받아 현재 '배송 상태' 문자열을 반환합니다.
        단, DB 연결 오류 시 "DB Error: {오류 메시지}" 반환하고,
        주문 ID가 없을 시 "Order Not Found"를 반환합니다.
    """
    # 디버깅용 (Agent가 이 함수를 호출할 때마다 콘솔에 출력됨)
    print(f"--- get_order_status_from_db 호출됨: {order_id} ---")
    try:
        # 0-3에서 만든 DB 파일에 연결
        conn = sqlite3.connect(base_path + 'orders.db')
        c = conn.cursor()
        # SQL 쿼리 실행: orders 테이블에서 order_id가 일치하는 row의 'status' 열을 선택
        c.execute("SELECT status FROM orders WHERE order_id = ?", (order_id,))
        # 결과 중 첫 번째 줄을 가져옴 (예: ('Delivered',))
        result = c.fetchone()
        conn.close()

        if result:
            return result[0] # 튜플이 아닌 문자열 값(예: 'Delivered')을 반환
        else:
            return "Order Not Found"
    except Exception as e:
        return f"DB Error: {str(e)}"

**도구 3: 배송 지연 쿠폰 발급 도구**

- "도구가 다른 도구를 호출"하는 복합적인 도구입니다.
- **로직:**
    1. `get_order_status_from_db` 도구를 **`.invoke()`** 로 호출하여 주문 상태를 먼저 확인.
    2. 상태가 "Shipping Delayed"일 때만 쿠폰 발급 (하드코딩).
- **(주의):** 도구 내에서 다른 도구를 호출할 때는 일반 함수처럼 `get_order_status_from_db(...)`로 호출하면 안 되고, 반드시 `.invoke(...)`를 사용해야 LangChain의 추적 시스템(LangSmith)이 올바르게 작동합니다.

In [ ]:
# 3. 배송 지연 쿠폰 발급 도구 함수
@tool
def issue_complaint_coupon(order_id: str) -> str:
    """(DB 조회 기반 쿠폰 발급)
        주문 ID를 받아, 배송 상태가 'Shipping Delayed'일 때만 쿠폰을 발급합니다.
        (실제 쿠폰 생성 로직은 하드코딩으로 대체하고, 발급 메시지만 반환합니다.)
    """
    # 디버깅용
    print(f"--- issue_complaint_coupon 호출됨: {order_id} ---")

    # (중요!) 도구가 다른 도구를 호출할 때는 .invoke() 사용
    order_status = get_order_status_from_db.invoke(order_id)

    # 비즈니스 로직: 배송 지연 상태가 맞는지 확인
    if order_status == "Shipping Delayed":
        # (원래라면 이곳에 쿠폰 DB INSERT 로직 등이 들어감)
        return f"주문 ID {order_id}에 대해 5,000원 할인 쿠폰이 발급되었습니다."
    else:
        return f"주문 ID {order_id}는 쿠폰 발급 대상이 아닙니다. 현재 상태: {order_status}"

----------

## 2. ReAct Agent 설계 및 실행

이제 '두뇌(LLM)'와 '손발(Tools)' 조립입니다.


### 2-1. ReAct (Reason + Act)란?

- LLM이 **추론(Reasoning)** 과 **행동(Acting)** 을 반복하며 복잡한 문제를 해결하는 프레임워크입니다.
- Agent는 사용자의 요청을 받으면, `Thought(생각) -> Action(행동) -> Observation(관찰)` 사이클을 반복합니다.


**예시: "내 주문 ORDER456 배송 지연됐던데 쿠폰 줘"**

1. **Thought (생각):** "고객이 ORDER456의 쿠폰을 원하네. 하지만 쿠폰은 '배송 지연' 상태일 때만 줘야 한다는 규칙이 있었지. 먼저 `get_order_status_from_db` 도구로 상태를 확인해야겠다."

2. **Action (행동):** `get_order_status_from_db[ORDER456]` 도구를 실행.

3. **Observation (관찰):** "도구 결과로 'Shipping Delayed'"라는 텍스트를 받음.

4. **Thought (생각):** "좋아, 'Shipping Delayed' 상태가 맞네. 이제 `issue_complaint_coupon` 도구를 사용해서 쿠폰을 발급하자."

5. **Action (행동):** `issue_complaint_coupon[ORDER456]` 도구를 실행.

6. **Observation (관찰):** "도구 결과로 '...5,000원 할인 쿠폰이 발급되었습니다.'"라는 텍스트를 받음.

7. **Thought (생각):** "모든 정보가 모였다. 이제 이 두 가지 결과를 조합해서 고객에게 최종 답변을 생성하자."

8. **Final Answer (최종 답변):** "네, 고객님. 주문...은 배송 지연으로 확인되어 5,000원 쿠폰이 발급되었습니다."

### 2-2. 시스템 프롬프트 설계 (Agent의 규칙)
- Agent가 이 ReAct 사이클을 따르고, 우리가 정한 '신뢰성(Trustworthiness) 규칙'을 지키도록 **시스템 프롬프트**로 명확하게 지시해야 합니다.
- 이것은 Agent의 '헌법' 또는 **'행동 강령'** 과 같습니다.

In [ ]:
system_prompt = """
당신은 AI 온라인 서점의 고객 서비스 AI 에이전트입니다.
당신은 사용자의 요청을 해결하기 위해 '도구(tool)'를 사용해야 합니다.

[작업 수행 가이드라인]
1.  사용자의 요청을 받으면, 먼저 '생각(Thought)'을 합니다.
2.  요청을 해결하는 데 필요한 정보가 무엇인지 분석합니다.
3.  필요한 정보를 수집하기 위해 어떤 '도구'를 사용해야 할지 결정합니다.
4.  도구를 사용한 후, '관찰(Observation)' 결과를 봅니다.
5.  모든 정보가 모일 때까지 1-4 단계를 반복합니다.
6.  모든 정보가 수집되면, 이를 바탕으로 사용자에게 최종 답변을 생성합니다.

[!!절대적 규칙!!]
- 'issue_complaint_coupon' 도구는 고객이 요청하더라도, 'get_order_status_from_db' 도구의 결과가 'Shipping Delayed'일 때만 사용해야 합니다.
- 주문 상태가 불명확하면, 쿠폰을 발급하기 전에 반드시 'get_order_status_from_db'를 먼저 사용해야 합니다.
- 절대 추측하지 말고, 항상 도구의 실행 결과를 바탕으로만 답변하세요.
- 정책에 대한 질문은 'Policy_Search' 도구를 사용하세요.
"""

### 2-3. Agent 생성 및 실행

- `langchain.agents.create_agent` 함수를 사용해 LLM(두뇌), Tools(손발), Prompt(규칙)를 결합합니다.

In [ ]:
from langchain.agents import create_agent
from langchain_upstage import ChatUpstage

# 1. Agent가 사용할 도구들을 리스트로 묶기
agent_tools = [retriever_tool, get_order_status_from_db, issue_complaint_coupon]

# 2. Agent의 '두뇌'가 될 LLM 정의
llm = ChatUpstage()

# 3. ReAct Agent 생성
# create_agent가 내부적으로 ReAct 프롬프트 템플릿과 LLM, Tools를 결합
agent = create_agent(
    model=llm,          # 사용할 LLM (두뇌)
    tools=agent_tools,  # 사용할 도구 리스트 (손발)
    system_prompt=system_prompt # 시스템 프롬프트 (규칙)
)
print("ReAct Agent 생성 완료")

ReAct Agent 생성 완료


### 2-4. Agent 테스트 및 로그 분석 (T-A-O)

- `agent.stream`을 사용하면 Agent의 '생각' 과정을 실시간으로 스트리밍하며 관찰할 수 있습니다.


#### 2-4-1. 시나리오 1: 배송 지연 + 쿠폰 요청 (규칙 준수 테스트)
- **요청:** "주문 ID ORDER456의 배송 상태가 어떻게 되나요? 만약 지연되었다면 쿠폰을 받을 수 있나요?"
- **예상 T-A-O:**
    1. **Thought:** (상태 확인 및 쿠폰 발급을 결정)

    2. **Action 1:** `get_order_status_from_db` 호출

    3. **Observation 1:** "Shipping Delayed"

    4. **Thought:** (지연 확인, 쿠폰 발급 결정)

    5. **Action 2:** `issue_complaint_coupon` 호출

    6. **Observation 2:** "쿠폰 발급됨"

    7. **Thought:** (최종 답변 생성)

    8. **Final Answer:** (최종 답변)


In [ ]:
query_1 = "주문 ID ORDER456의 배송 상태가 어떻게 되나요? 만약 지연되었다면 쿠폰을 받을 수 있나요?"

# agent.stream을 사용하여 Agent의 실시간 반응(T-A-O)을 관찰
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": query_1}]},
    stream_mode="updates", # 'updates' 모드는 ReAct의 각 '단계'별로만 청크를 반환
):
    print("--- CHUNK ---")
    # 청크의 content_blocks를 출력하여 T-A-O 과정을 확인
    for step, data in chunk.items():
        print(f"step: {step}")
        if 'messages' in data:
            # content_blocks에 Thought, Action(ToolCall), Observation(ToolResult), AiMessage가 포함됨
            pprint(f"content: {data['messages'][-1].content_blocks}")

--- CHUNK ---
step: model
("content: [{'type': 'tool_call', 'id': "
 "'4e332a93-bfac-4e70-be35-9f733aa57728', 'name': 'get_order_status_from_db', "
 "'args': {'order_id': 'ORDER456'}}]")
--- get_order_status_from_db 호출됨: ORDER456 ---
--- CHUNK ---
step: tools
"content: [{'type': 'text', 'text': 'Shipping Delayed'}]"
--- CHUNK ---
step: model
("content: [{'type': 'tool_call', 'id': "
 "'0cd938de-07c0-4d82-baca-869d576af3ac', 'name': 'issue_complaint_coupon', "
 "'args': {'order_id': 'ORDER456'}}]")
--- issue_complaint_coupon 호출됨: ORDER456 ---
--- get_order_status_from_db 호출됨: ORDER456 ---
--- CHUNK ---
step: tools
("content: [{'type': 'text', 'text': '주문 ID ORDER456에 대해 5,000원 할인 쿠폰이 "
 "발급되었습니다.'}]")
--- CHUNK ---
step: model
('content: [{\'type\': \'text\', \'text\': "주문 ID ORDER456의 배송 상태는 현재 '
 '\'Shipping Delayed\'입니다. 이에 따라, 5,000원 할인 쿠폰이 발급되었습니다."}]')


#### 2-4-2. 시나리오 2: 배송 완료 + 쿠폰 요청 (규칙 거부 테스트)
- **요청:** "제 주문 ORDER123인데요. 쿠폰 좀 발급해주세요."

- **예상 T-A-O:**

    1. Thought: (쿠폰 발급 전 상태 확인 결정)

    2. Action 1: `get_order_status_from_db` 호출

    3. Observation 1: "Delivered"

    4. Thought: ('Delivered'이므로 쿠폰 발급 불가 결정, `issue_complaint_coupon` 호출 안 함)

    5. Final Answer: "고객님, 주문 ORDER123은 'Delivered'(배송 완료) 상태로 확인되어 쿠폰 발급 대상이 아닙니다."

In [ ]:
query_2 = "제 주문 ORDER123인데요. 쿠폰 좀 발급해주세요."

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": query_2}]},
    stream_mode="updates",
):
    print("--- CHUNK ---")
    for step, data in chunk.items():
        print(f"step: {step}")
        if 'messages' in data:
            print(f"content: {data['messages'][-1].content_blocks}")

--- CHUNK ---
step: model
content: [{'type': 'tool_call', 'id': '175aca73-a206-470f-8a0e-662ef24a428a', 'name': 'get_order_status_from_db', 'args': {'order_id': 'ORDER123'}}]
--- get_order_status_from_db 호출됨: ORDER123 ---
--- CHUNK ---
step: tools
content: [{'type': 'text', 'text': 'Delivered'}]
--- CHUNK ---
step: model
content: [{'type': 'text', 'text': "주문 상태는 'Delivered'로 확인되었습니다. 따라서 현재 주문 상태로는 쿠폰을 발급할 수 없습니다. 다른 도움이 필요하신가요?"}]


## Extra. Short-term Memory (대화 기억)
- 지금까지 만든 Agent는 **'기억력(Memory)'이 없습니다.** 각 `stream` 또는 `invoke` 호출은 완전히 새로운 대화입니다.
- **문제점:**
    - **User:** "배송 조회하려면 뭐가 필요해?"
    - **Agent:** "주문 ID가 필요합니다."
    - **User:** "ORDER456이야."
    - **Agent:** (기억 못 함) "무엇을 도와드릴까요?"
- 해결책: `LangGraph`의 **Checkpointer**를 사용하여 대화 기록을 DB에 저장합니다.

### E-1. LangGraph Checkpointer 설치
- Agent의 '상태(State)'와 '대화 기록(Messages)'을 관리하고 DB에 저장(Checkpoint)하기 위한 `langgraph-checkpoint-sqlite` 라이브러리를 설치합니다.

- (참고: `LangGraph`는 Agent의 흐름을 '그래프(Graph)'로 설계하는 고급 라이브러리입니다. 오늘은 이 중 '메모리 저장' 기능만 간단히 사용해 봅니다.)

In [ ]:
# Agent의 대화 기록을 SQLite DB에 저장하기 위한 라이브러리
!pip install langgraph-checkpoint-sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 9.5 MB/s eta 0:00:00


### E-2. Agent에 Checkpointer(메모리) 적용
#### E-2-1. Checkpointer 정의
- `SqliteSaver`: 대화 기록(Checkpoint)을 SQLite DB 파일(`database.db`)에 저장하는 객체입니다.
- `AgentState`: Agent가 기억해야 할 대화 상태의 '구조(Schema)'를 정의합니다. (기본적으로 messages 포함)

In [ ]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

# 1. 대화 기록을 저장할 DB(database.db)와 연결하는 Checkpointer 생성
# (check_same_thread=False는 sqlite가 여러 스레드에서 접근 가능하도록 허용하는 설정)
checkpointer = SqliteSaver(
    sqlite3.connect(base_path + "database.db", check_same_thread=False)
)

# 2. (선택) Agent가 'messages' 외에 추가로 기억할 상태 정의
# class CustomAgentState(AgentState):
#     user_id: str # 예: 사용자 ID
#     preferences: dict # 예: 사용자 선호도

# 3. Agent를 다시 생성하되, 'checkpointer' 옵션을 추가
agent_with_memory = create_agent(
    model=llm,
    tools=agent_tools,
    system_prompt=system_prompt,
    # state_schema=CustomAgentState, # (만약 CustomAgentState를 쓴다면)
    checkpointer=checkpointer # (중요!) Agent에 메모리 시스템 장착
)
print("메모리가 적용된 Agent 생성 완료")

NameError: name 'base_path' is not defined

#### E-2-2. RunnableConfig 설정 (대화 ID 지정)
- Agent에게 "이 대화는 몇 번 방(Thread)에 저장해 줘"라고 알려줘야 합니다.

- `RunnableConfig`의 `{"configurable": {"thread_id": "..."}}`가 이 역할을 합니다.

- **동일한 `thread_id`** 를 사용하는 모든 `invoke` 호출은 **같은 대화 기록**을 공유합니다.

In [ ]:
from langchain_core.runnables import RunnableConfig

# '1번 고객'과의 대화방을 설정 (이 ID를 바꾸면 다른 대화방이 됨)
config: RunnableConfig = {"configurable": {"thread_id": "1"}}

#### E-2-3. 대화가 이어지는지 테스트
- **동일한 `config`** 를 사용해서 Agent를 여러 번 호출해 봅니다.

In [ ]:
# 첫 번째 질문
# (Agent는 이 질문과 답변을 config의 'thread_id': '1'에 저장합니다)
print("--- User (Query 1) ---")
result1 = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": '배송 조회 하려면 뭐가 필요해?'}]},
    config
)
result1['messages'][-1].pretty_print()

In [ ]:
# 두 번째 질문 (이전 대화를 기억해야 함)
# Agent는 'thread_id': '1'에서 이전 대화("주문 ID가 필요합니다")를 로드합니다.
# 그리고 "ORDER456"이 그 주문 ID임을 '맥락(Context)' 속에서 이해합니다.
print("\n--- User (Query 2) ---")
result2 = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": '주문 ID ORDER456 이래'}]},
    config
)
result2['messages'][-1].pretty_print()

In [ ]:
# 세 번째 질문 (방금 전 맥락을 기억해야 함)
# Agent는 'thread_id': '1'에서 방금 쿠폰을 발급한 사실을 기억합니다.
# "왜 주는지" 묻자, 메모리를 뒤져보고 "Shipping Delayed" 때문임을 인지한 뒤,
# 'Policy_Search' 도구를 호출하여 '배송 지연 정책'을 찾아 답변합니다.
print("\n--- User (Query 3) ---")
result3 = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": '뭐야 쿠폰 왜 주는거야?'}]},
    config
)
result3['messages'][-1].pretty_print()

In [ ]:
# (참고) 'thread_id' = '1'에 저장된 전체 대화 기록 확인
# checkpointer에 저장된 모든 대화 내용을 볼 수 있습니다.
print("\n--- 'thread_id=1'의 전체 대화 기록 ---")
history = checkpointer.get(config)
for message in history['channel_values']['messages']:
    message.pretty_print()